In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation              
from scipy.fftpack import fft2,ifft2 
from matplotlib import cm
import types

In [2]:
def gaussian_wavepack(x, y, sigma_x, sigma_y, x0, y0, k0_x, k0_y):
    """Creates a Gaussian wavepack with momentum k0_x*hbar and width sigma_x in the x-direction.
    The y-direction is specified by the similar variables k0_y, sigma_y"""
    return ((np.sqrt(sigma_x**2 + sigma_y**2) * (np.pi)) ** (-0.5) * 
            np.exp(-0.5 * ((x - x0) * 1. / sigma_x) ** 2 - 0.5 * ((y - y0) * 1. / sigma_y) ** 2 + 1j * (x * k0_x+ y* k0_y)))

def gaussian_planewave_x(x, sigma_x, x0, k0_x):
    """Creates a Gaussian planewave propagating in the x-direction with momentum k0*hbar and width sigma."""
    return ((sigma_x * (np.pi)) ** (-0.5) * np.exp(-0.5 * ((x - x0) * 1. / sigma_x) ** 2 + 1j * x * k0_x))

def potential_slit(N_x, N_y,a,d,V0,width,number_of_slits):
    """
    Creates a potential barrier in the y-direction of height V0 on N_x in the x-direction.
    The barrier has 'number_of_slits' slits with width 2a spaced 2d apart. The middle point between 
    the slits is located at N_y//2. In case number_of_slits is odd, an additional slit is created at N_y//2
        Args:
        N_x, N_y : float
            Size of the domain of the potential
        a : float
            vertical distance from the middle to the edge of the slit
        d : float
            half vertical distance between the slits
        width: float
            width of the potential barrier
        number_of_slits : float
            number of slits in the potential barrier
    """
    V = np.zeros((N_x,N_y))
    V[:,N_x//2 - width:N_x//2 + width] = V0
    if number_of_slits % 2 != 0:
        V[N_y//2 - a:N_y//2 + a,:] = 0          
    for i in range(number_of_slits//2):
        V[N_y//2 - a+(2*i+1)*d:N_y//2 + a+(2*i+1)*d,:] = 0
        V[N_y//2 - a-(2*i+1)*d:N_y//2 + a-(2*i+1)*d,:] = 0           
    return V

def tilted_potential_slit(N_x, N_y,a,d,V0,width,number_of_slits):
    """
    Creates a tilted potential barrier of height V0 trough the whole y-direction starting from the middle of the x-axis.
    The middle point between the slits is located at N_y//2. In case number_of_slits is odd, an additional slit is created 
    at N_y//2. Commenting the lower part imposes periodic BC on the edges of the domain.
    Args:
        N_x, N_y : float
            Size of the domain of the potential
        a : float
            vertical distance from the middle to the edge of the slit
        d : float
            half vertical distance between the slits
        width: float
            width of the potential barrier
        number_of_slits : float
            number of slits in the potential barrier
    """
    V = np.zeros((N_x,N_y))      
    for i in range(N_x):
        V[i:i+1,(N_x+i)//2] = V0
    for i in range(number_of_slits//2):
        V[N_y//2 - a+(2*i+1)*d:N_y//2 + a+(2*i+1)*d,:] = 0
        V[N_y//2 - a-(2*i+1)*d:N_y//2 + a-(2*i+1)*d,:] = 0 
    if number_of_slits % 2 != 0:
        V[N_y//2 - a:N_y//2 + a,:] = 0
    
    #set potential wall on edges domain
    V[0,:] = V0
    V[:,0] = V0
    V[N_y-1,:] = V0
    V[:,N_x-1] = V0
    return V

In [7]:
class WaveFunction(object):
    def __init__(self, x, y, psi_r0, V, hbar, m, t0, dt, k_range_x=0, k_range_y=0):        
        """
        Args:
            x, y : array, float
                The domain of computations, a vector with x-coordinates.
            psi_r0 : array, complex
                The initial wavefunction, a vector with complex number corresponding to each position.
            V : array, float
                The potential, an array with a float number corresponding to the potential at each coordinate
            hbar : float
                The value of the plank's constant, default 1
            m : float
                Mass of the particle, default 1
            t0 : float
                inital time, default 0
            dt : float
                The value of the timestep, default 0.01
            k_range_x, k_range_y : float
                These values specify the range of the k_x and k_y values. For the default k_range = 0 a suitable range is
                used: [-0.5*N*dk,0.5*N*dk]
                Otherwise the range [-k_range, -k_range + N*dk]  is used. This might be useful if you know that wavenumbers
                outside of the default range will be non-zero.
        """
        #set internal params
        self.x = x
        self.y = y
        self.V = V
        self.N_x = len(x)
        self.N_y = len(y)
        self.t = t0
        self.m = m
        self.hbar = hbar    
        self.dx = self.x[1] - self.x[0]
        self.dy = self.y[1] - self.y[0]       
        self.dk_x = 2 * np.pi / (self.N_x * self.dx)
        self.k_x =  -0.5 * self.N_x * self.dk_x + self.dk_x * np.arange(self.N_x)
        
        if k_range_x != 0:
            self.k_x = -k_range_x + self.dk_x * np.arange(self.N_x)
            
        self.dk_y = 2 * np.pi / (self.N_y * self.dy)
        self.k_y =  -0.5 * self.N_y * self.dk_y + self.dk_y * np.arange(self.N_y)
        
        if k_range_y != 0:
            self.k_y = -k_range_y + self.dk_y * np.arange(self.N_y)
        
        self.k_xx, self.k_yy = np.meshgrid(self.k_x,self.k_y)
        self.xx, self.yy = np.meshgrid(self.x,self.y)
        
        #set initial psi_r and psi_k
        self.psi_r = psi_r0
        self.fft()
        
        #set evolution operators
        self.r_evolution_half = np.exp(-0.5 * 1j * self.V/ self.hbar * dt )
        self.k_evolution = np.exp(-0.5 * 1j * self.hbar/self.m * (self.k_xx * self.k_xx + self.k_yy * self.k_yy) * dt)
        
    #define functions for getters and setters and combine them in a property
    def _set_psi_r(self, psi_r):
        self.psi_fou_r = (psi_r * np.exp(-1j * (self.k_x[0] * self.xx + self.k_y[0] * self.yy)) \
                          * (self.dx / np.sqrt(2 * np.pi)) * (self.dy / np.sqrt(2 * np.pi)))
    def _get_psi_r(self):
        return (self.psi_fou_r * np.exp(1j * (self.k_x[0] * self.xx + self.k_y[0] * self.yy)) \
                * (np.sqrt(2 * np.pi) / self.dx) * (np.sqrt(2 * np.pi) / self.dy))

    def _set_psi_k(self, psi_k):
        self.psi_fou_k = psi_k * np.exp(1j * (self.x[0]* self.dk_x * np.arange(self.N_x) \
                                              + self.y[0]* self.dk_y * np.arange(self.N_y)))
    def _get_psi_k(self):
        return self.psi_fou_k * np.exp(-1j * (self.x[0]* self.dk_x * np.arange(self.N_x) \
                                              + self.y[0]* self.dk_y * np.arange(self.N_y)))
           
    psi_r = property(_get_psi_r, _set_psi_r)
    psi_k = property(_get_psi_k, _set_psi_k) 
    
    #define FFT and iFFT
    def fft(self):
        self.psi_fou_k = fft2(self.psi_fou_r)
        
    def ifft(self):
        self.psi_fou_r = ifft2(self.psi_fou_k)
    
    #calculate a timestep
    def time_step(self, dt, steps):
        for i in range(steps - 1):
            self.psi_fou_r *= self.r_evolution_half
            self.fft()
            self.psi_fou_k *= self.k_evolution
            self.ifft()
            self.psi_fou_r *= self.r_evolution_half

        self.fft()
        self.t += dt * steps

In [8]:
#set-up simulation
"""Specify the input params of the simulation. To toggle between tilted and normal potential barrier and planewave gaussian 
or circular gaussian, comment and uncomment the relevant lines."""
#specify simple params
hbar = 1          #Planck's constant /2 pi
m = 1.9           #mass particle
t0 = 0            #starting time
dt = 0.01         #length of time-step
steps = 10        #number of timesteps between each animated frame
frames = 300      

#specify space-range
N_x = 200
dx = 0.1
x = dx * (np.arange(N_x) - 0.5 * N_x)
N_y = 200
dy = 0.1
y = dy * (np.arange(N_y) - 0.5 * N_y)
xx, yy = np.meshgrid(x, y)
dx = x[1]-x[0]
dy = y[1]-y[0]

#specify initial wave
#psi_r0 = gaussian_wavepack(xx, yy, sigma_x = 1, sigma_y = 1, x0 = -8, y0 = 0, k0_x = 5, k0_y = 0)
psi_r0 = gaussian_planewave_x(xx, sigma_x = 1, x0 = -3, k0_x = 5)

#specify potential
V0 = 1e10    #potential height
d = 12       #distance between slits
a = 8        #width slit
width = 1    #width potential barrier
number = 6   #number of slits
V = potential_slit(N_x, N_y, a, d, V0, width, number)
#V = tilted_potential_slit(N_x, N_y, a, d, V0, width, number)


#create instance of 
B = WaveFunction(x=x, y=y, psi_r0=psi_r0, V=V, hbar=hbar, m=m, t0=0.0, dt=dt, k_range_x=0, k_range_y=0)

In [9]:
#set-up animation
xx, yy = np.meshgrid(B.x, B.y)
fig = plt.figure()

xlim = [B.x[0],B.x[-1]]
ylim = [B.y[0],B.y[-1]]
plot1 = fig.add_subplot(211, xlim=xlim, ylim=ylim) 
plot1.set_xlabel(r'$x$-coordinate')
plot1.set_ylabel(r'$y$-coordinate')

kx, ky = np.meshgrid(B.k_x, B.k_y)
kxlim = [B.k_x[0],B.k_x[-1]]
kylim = [B.k_y[0],B.k_y[-1]]
plot2 = fig.add_subplot(212, xlim=kxlim, ylim=kylim)
plot2.set_xlabel(r'$k_x$')
plot2.set_ylabel(r'$k_y$')

#animation function
def animate(i):
    B.time_step(dt, steps)
    R = plot1.contourf(xx, yy, abs(B.psi_r), 30, label=r'$|\psi(x)|$')
    K = plot2.contourf(kx, ky, abs(B.psi_k), 30, label = r'$| \psi(k) |$')
    return R 

anim = animation.FuncAnimation(fig, animate, frames=frames)
plt.show()